## 3. Read Mapping

*Tools*

**Bowtie2**
* Conda: https://bioconda.github.io/recipes/bowtie2/README.html#package-bowtie2
* Manual: http://bowtie-bio.sourceforge.net/bowtie2/index.shtml

**bwa**
* Conda: https://bioconda.github.io/recipes/bwa/README.html#package-bwa
* Manual: http://bio-bwa.sourceforge.net/bwa.shtml

**bwa-mem2** 
* Conda: https://bioconda.github.io/recipes/bwa-mem2/README.html#package-bwa-mem2
* Manual: https://www.biostars.org/p/301324/

**samtools**
* Conda: https://bioconda.github.io/recipes/samtools/README.html#package-samtools
* Manual: http://www.htslib.org/doc/

* To make it work: `conda update --all`

** **

*SAM format* 

https://samtools.github.io/hts-specs/SAMv1.pdf

Meaning of columns https://www.samformat.info/sam-format-flag

`QNAME    FLAG    RNAME    POS    MAPQ    CIGAR    RNEXT    PNEXT    TLEN    SEQ    QUAL    TAGS`
  1. Read Name
  2. SAM flag https://broadinstitute.github.io/picard/explain-flags.html
  3. contig name, reference sequence name, RNAME, often contains the Chromosome name or * for unmapped
  4. mapped position of base 1 of a read on the reference sequence
  5. MAPQ mapping quality: phred-scaled posterior probability that the mapping position is wrong  
      *It equals −10 log10 Pr{mapping position is wrong}, rounded to the nearest integer. A value 255 indicates that the mapping quality is not available*
  6. CIGAR string describing insertions and deletions
  7. The reference sequence name of the next alignment in this group, MRNM or RNEXT. In paired alignments, it is the mate's reference sequence name. (A group is alignments with the same query name.)
  8. Position of mate
  9. Template length
  10. Read Sequence
  11. Read Quality
  12. Additional information in `TAG:TYPE:VALUE` format
  
https://genome.sph.umich.edu/wiki/SAM

** ** 

*Steps to be performed*

1. **Index the reference sequence**. Each reference/genome only needs to be indexed once for each mapper. Each mapper has it own command to index the reference.
2. **Map the reads**. Usually the mappers return a file in sam format.
3. If the output of the mapper is a sam, **convert it to bam**.
4. **Sort the bam**
5. **Index the bam**

** ** 
*Bowtie2*

In [2]:
! ls ./P1/

Archive          brca.fasta       brca_reads.fastq


In [5]:
# 1. Index the reference
! bowtie2-build ./P1/brca.fasta ./P1/brca.index

Settings:
  Output files: "./P1/brca.index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ./P1/brca.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 1841
Using parameters --bmax 1381 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 1381 --dcv 1024
Constructing suffix-array ele

In [6]:
! ls ./P1/

Archive              brca.index.2.bt2     brca.index.rev.1.bt2
brca.fasta           brca.index.3.bt2     brca.index.rev.2.bt2
brca.index.1.bt2     brca.index.4.bt2     brca_reads.fastq


In [7]:
# 2. Map the reads
! bowtie2 -sensitive -x ./P1/brca.index -U ./P1/brca_reads.fastq -S ./P1/bowtie2_alignment.sam

6 reads; of these:
  6 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    5 (83.33%) aligned exactly 1 time
    1 (16.67%) aligned >1 times
100.00% overall alignment rate


In [8]:
## Add -a option: search and report all alignments
! bowtie2 -sensitive -a -x ./P1/brca.index -U ./P1/brca_reads.fastq -S ./P1/bowtie2_alignment2.sam

6 reads; of these:
  6 (100.00%) were unpaired; of these:
    0 (0.00%) aligned 0 times
    5 (83.33%) aligned exactly 1 time
    1 (16.67%) aligned >1 times
100.00% overall alignment rate


In [9]:
# 3. Convert to bam
! samtools view -hb -o ./P1/bowtie2_alignment.bam -S ./P1/bowtie2_alignment.sam

In [10]:
! ls ./P1/

Archive                brca.fasta             brca.index.4.bt2
bowtie2_alignment.bam  brca.index.1.bt2       brca.index.rev.1.bt2
bowtie2_alignment.sam  brca.index.2.bt2       brca.index.rev.2.bt2
bowtie2_alignment2.sam brca.index.3.bt2       brca_reads.fastq


In [11]:
# 4. Sort the bam 
! samtools sort ./P1/bowtie2_alignment.bam -o ./P1/bowtie2_alignment.sorted.bam

In [12]:
! ls ./P1/

Archive                      brca.index.2.bt2
bowtie2_alignment.bam        brca.index.3.bt2
bowtie2_alignment.sam        brca.index.4.bt2
bowtie2_alignment.sorted.bam brca.index.rev.1.bt2
bowtie2_alignment2.sam       brca.index.rev.2.bt2
brca.fasta                   brca_reads.fastq
brca.index.1.bt2


In [13]:
# 5. Index the bam
! samtools index ./P1/bowtie2_alignment.sorted.bam

In [14]:
! ls ./P1/

Archive                          brca.index.1.bt2
bowtie2_alignment.bam            brca.index.2.bt2
bowtie2_alignment.sam            brca.index.3.bt2
bowtie2_alignment.sorted.bam     brca.index.4.bt2
bowtie2_alignment.sorted.bam.bai brca.index.rev.1.bt2
bowtie2_alignment2.sam           brca.index.rev.2.bt2
brca.fasta                       brca_reads.fastq


** ** 
*bwa*

In [17]:
! ls ./P3/

Archive                 maize_unigene_ref.fasta root.454.fastq


In [18]:
# 1. Index the reference 
! bwa index -p reference -a bwtsw ./P3/maize_unigene_ref.fasta

[bwa_index] Pack FASTA... 0.01 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=614110, availableWord=780628
[bwt_gen] Finished constructing BWT in 5 iterations.
[bwa_index] 0.10 seconds elapse.
[bwa_index] Update BWT... 0.01 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.03 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index -p reference -a bwtsw ./P3/maize_unigene_ref.fasta
[main] Real time: 0.251 sec; CPU: 0.162 sec


In [21]:
# Had to manually move the files in the folder as they were created in the parent directory
! ls ./P3/

Archive                 reference.ann           reference.sa
maize_unigene_ref.fasta reference.bwt           root.454.fastq
reference.amb           reference.pac


In [22]:
# 2. Map the reads 
! bwa mem ./P3/reference ./P3/root.454.fastq > ./P3/mapped_res.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 4888 sequences (1034913 bp)...
[M::mem_process_seqs] Processed 4888 reads in 0.900 CPU sec, 0.904 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem ./P3/reference ./P3/root.454.fastq
[main] Real time: 1.234 sec; CPU: 0.970 sec


In [23]:
# 3. Convert to bam
# 4. Sort bam
# 5. Index ba

## to be performed with samtools as above

** ** 
*Convert reference input file from genebank to fasta using python script*

**Biopython:** https://anaconda.org/anaconda/biopython
**SeqIO:** https://biopython.org/wiki/SeqIO

In [1]:
! ls ./P4/

Archive                     mito_reads.tar.gz
mito_human_ref_nc_012920.gb


In [2]:
from Bio import SeqIO
infile_genbank = "./P4/mito_human_ref_nc_012920.gb"
outfile = "./P4/mito_human_ref_nc_012920.fasta"

SeqIO.convert(infile_genbank, "genbank", outfile, "fasta")

1

In [3]:
! ls ./P4/

Archive                        mito_human_ref_nc_012920.gb
mito_human_ref_nc_012920.fasta mito_reads.tar.gz


In [4]:
# 1. Index the reference
! bowtie2-build ./P4/mito_human_ref_nc_012920.fasta ./P4/mito_human_index

Settings:
  Output files: "./P4/mito_human_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ./P4/mito_human_ref_nc_012920.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 4142
Using parameters --bmax 3107 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 3107 --dcv 1024
Con

In [5]:
! ls ./P4/

Archive                        mito_human_index.rev.1.bt2
mito_human_index.1.bt2         mito_human_index.rev.2.bt2
mito_human_index.2.bt2         mito_human_ref_nc_012920.fasta
mito_human_index.3.bt2         mito_human_ref_nc_012920.gb
mito_human_index.4.bt2         mito_reads.tar.gz


In [7]:
! tar -xvzf ./P4/mito_reads.tar.gz

x mito_reads_pe1.fastq.gz
x mito_reads_pe2.fastq.gz


In [9]:
# Manually move the reads in the folder
! ls ./P4/

Archive                        mito_human_index.rev.2.bt2
mito_human_index.1.bt2         mito_human_ref_nc_012920.fasta
mito_human_index.2.bt2         mito_human_ref_nc_012920.gb
mito_human_index.3.bt2         mito_reads.tar.gz
mito_human_index.4.bt2         mito_reads_pe1.fastq.gz
mito_human_index.rev.1.bt2     mito_reads_pe2.fastq.gz


In [25]:
# 2. Mapping the reads - paired end
! bowtie2 --very-fast -x ./P4/mito_human_index -1 ./P4/mito_reads_pe1.fastq.gz -2 ./P4/mito_reads_pe2.fastq.gz -S ./P4/1.aligned_reads.sam


10000 reads; of these:
  10000 (100.00%) were paired; of these:
    5101 (51.01%) aligned concordantly 0 times
    4899 (48.99%) aligned concordantly exactly 1 time
    0 (0.00%) aligned concordantly >1 times
    ----
    5101 pairs aligned concordantly 0 times; of these:
      4581 (89.81%) aligned discordantly 1 time
    ----
    520 pairs aligned 0 times concordantly or discordantly; of these:
      1040 mates make up the pairs; of these:
        737 (70.87%) aligned 0 times
        303 (29.13%) aligned exactly 1 time
        0 (0.00%) aligned >1 times
96.31% overall alignment rate


Explanation: 

A pair that aligns with the expected relative mate orientation and with the expected range of distances between mates is said to align **concordantly**. If both mates have unique alignments, but the alignments do not match paired-end expectations (i.e. the mates aren’t in the expected relative orientation, or aren’t within the expected distance range, or both), the pair is said to align **discordantly**.

* aligned 0 times: unmapped
* aligned exactly 1 times: unique match
* aligned >1 times: multiple match

Above
* Total reads (paired): *10.000*
  * Aligned concordantly: *4899 (48.99%)*
  * Remaining: *5101 (51.01%)*
    * Aligned discordantly: *4581 (89.81%)*
    * Remaining: *520* -> pairs divided: *1040* reads
      * Aligned: *303 (29.13%)*
      * Remaining: *737 (70.87%)*
* Overall aligment rate: *96.31% (4899+4581+(303/2)*)
* Reads not aligned: *737*

In [26]:
# Try more sensitive setting
! bowtie2 --very-sensitive-local -x ./P4/mito_human_index -1 ./P4/mito_reads_pe1.fastq.gz -2 ./P4/mito_reads_pe2.fastq.gz -S ./P4/1.aligned_reads.sam


10000 reads; of these:
  10000 (100.00%) were paired; of these:
    5001 (50.01%) aligned concordantly 0 times
    4999 (49.99%) aligned concordantly exactly 1 time
    0 (0.00%) aligned concordantly >1 times
    ----
    5001 pairs aligned concordantly 0 times; of these:
      4895 (97.88%) aligned discordantly 1 time
    ----
    106 pairs aligned 0 times concordantly or discordantly; of these:
      212 mates make up the pairs; of these:
        166 (78.30%) aligned 0 times
        46 (21.70%) aligned exactly 1 time
        0 (0.00%) aligned >1 times
99.17% overall alignment rate


More reads aligned to reference with `--very-sensitive-local` settings (*99.17%* vs *96.31%*)

`--very-fast`: `-D 5 -R 1 -N 0 -L 22 -i S,0,2.50`

`--very-sensitive`: `-D 20 -R 3 -N 0 -L 20 -i S,1,0.50`

**Effort options**

`-D INT`: up to INT consecutive seed extension attempts can “fail” before Bowtie 2 moves on, using the alignments found so far
* A seed extension “fails” if it does not yield a new best or a new second-best alignment
* This limit is automatically adjusted up when -k or -a are specified
* Default: 15

`-R INT`: INT is the maximum number of times Bowtie 2 will “re-seed” reads with repetitive seeds
* When “re-seeding,” Bowtie 2 simply chooses a new set of reads (same length, same number of mismatches allowed) at different offsets and searches for more alignments
* A read is considered to have repetitive seeds if the total number of seed hits divided by the number of seeds that aligned at least once is greater than 300
* Default: 2

**Aligment options**
    
`-N INT`: sets the number of mismatches to allowed in a seed alignment during multiseed alignment 
* Can be set to 0 or 1
* Setting this higher makes alignment slower (often much slower) but increases sensitivity
* Default: 0

`-L INT`: sets the length of the seed substrings to align during multiseed alignment
* Smaller values make alignment slower but more sensitive
* Default: the `--sensitive` preset is used by default, which sets `-L` to `22` and `20` in `--end-to-end` mode and in `--local` mode

`-i FUNC`: sets a function governing the interval between seed substrings to use during multiseed alignment
* For instance, if the read has 30 characters, and seed length is 10, and the seed interval is 6, the seeds extracted will be:

`Read:      TAGCTACGCTCTACGCTATCATGCATAAAC
Seed 1 fw: TAGCTACGCT
Seed 1 rc: AGCGTAGCTA
Seed 2 fw:       CGCTCTACGC
Seed 2 rc:       GCGTAGAGCG
Seed 3 fw:             ACGCTATCAT
Seed 3 rc:             ATGATAGCGT
Seed 4 fw:                   TCATGCATAA
Seed 4 rc:                   TTATGCATGA`


* Since it’s best to use longer intervals for longer reads, this parameter sets the interval as a function of the read length, rather than a single one-size-fits-all number
* For instance, specifying `-i S,1,2.5` sets the interval function f to `f(x) = 1 + 2.5 * sqrt(x)`, where `x` is the read length
* If the function returns a result less than 1, it is rounded up to 1
* Default: the `--sensitive` preset is used by default, which sets `-i` to `S,1,1.15` in `--end-to-end` mode to `-i S,1,0.75` in `--local` mode
    
**Reporting options**
    
`-k INT`
* By default, bowtie2 searches for distinct, valid alignments for each read. When it finds a valid alignment, it continues looking for alignments that are nearly as good or better. The best alignment found is reported (randomly selected from among best if tied). Information about the best alignments is used to estimate mapping quality and to set SAM optional fields, such as `AS:i` and `XS:i`.
* When -k is specified, bowtie2 behaves differently. Instead, it searches for at most INT distinct, valid alignments for each read. The search terminates when it can’t find more distinct valid alignments, or when it finds INT, whichever happens first. All alignments found are reported in descending order by alignment score. 
  * The alignment score for a paired-end alignment equals the sum of the alignment scores of the individual mates. Each reported read or pair alignment beyond the first has the SAM ‘secondary’ bit (which equals 256) set in its FLAGS field. 
  * For reads that have more than INT distinct, valid alignments, bowtie2 does not guarantee that the INT alignments reported are the best possible in terms of alignment score
* `-k` is mutually exclusive with `-a`

***Note:*** *Bowtie 2 is not designed with large values for -k in mind, and when aligning reads to long, repetitive genomes large -k can be very, very slow.*

`-a`
* Like -k but with no upper limit on number of alignments to search for
* `-a` is mutually exclusive with `-k`.

***Note:*** *Bowtie 2 is not designed with -a mode in mind, and when aligning reads to long, repetitive genomes this mode can be very, very slow.*

**Sam optional fields** 

`AS:i:N` Alignment score
* Can be negative
* Can be greater than 0 in `--local` mode (but not in `--end-to-end` mode)
* Only present if SAM record is for an aligned read

`XS:i:N` Alignment score for the best-scoring alignment found other than the alignment reported
* Can be negative
* Can be greater than 0 in `--local` mode (but not in `--end-to-end` mode)
* Only present if the SAM record is for an aligned read and more than one alignment was found for the read
* Note that, when the read is part of a concordantly-aligned pair, this score could be greater than `AS:i`
    
**Theory**

* **Alignment**: https://www.sevenbridges.com/short-read-alignment-algorithms/?__hstc=64521200.b81c63075247264fc44c70adb897a974.1595904494433.1595904494433.1595904494433.1&__hssc=64521200.1.1595904494434&__hsfp=2365945144

In [27]:
# 3. Convert to bam
! samtools view -hb -o ./P4/2.aligned_reads.bam -S ./P4/1.aligned_reads.sam

In [28]:
! ls ./P4/

1.aligned_reads.sam            mito_human_index.rev.1.bt2
2.aligned_reads.bam            mito_human_index.rev.2.bt2
Archive                        mito_human_ref_nc_012920.fasta
mito_human_index.1.bt2         mito_human_ref_nc_012920.gb
mito_human_index.2.bt2         mito_reads.tar.gz
mito_human_index.3.bt2         mito_reads_pe1.fastq.gz
mito_human_index.4.bt2         mito_reads_pe2.fastq.gz


In [29]:
# 4. Sort bam
! samtools sort ./P4/2.aligned_reads.bam -o ./P4/3.sorted_alignments.bam

In [30]:
! ls ./P4/

1.aligned_reads.sam            mito_human_index.rev.1.bt2
2.aligned_reads.bam            mito_human_index.rev.2.bt2
3.sorted_alignments.bam        mito_human_ref_nc_012920.fasta
Archive                        mito_human_ref_nc_012920.gb
mito_human_index.1.bt2         mito_reads.tar.gz
mito_human_index.2.bt2         mito_reads_pe1.fastq.gz
mito_human_index.3.bt2         mito_reads_pe2.fastq.gz
mito_human_index.4.bt2


In [31]:
# 5. Index bam
! samtools index ./P4/3.sorted_alignments.bam

In [32]:
! ls ./P4/

1.aligned_reads.sam            mito_human_index.4.bt2
2.aligned_reads.bam            mito_human_index.rev.1.bt2
3.sorted_alignments.bam        mito_human_index.rev.2.bt2
3.sorted_alignments.bam.bai    mito_human_ref_nc_012920.fasta
Archive                        mito_human_ref_nc_012920.gb
mito_human_index.1.bt2         mito_reads.tar.gz
mito_human_index.2.bt2         mito_reads_pe1.fastq.gz
mito_human_index.3.bt2         mito_reads_pe2.fastq.gz


** ** 

*Filtering using samtools*

Options for filtering: 
* Read group
* Flag
* Mapping quality
* Genome location

In [38]:
# Filter reads located in the first 10 kilobases of the mithocondrial genome that have a mapping quality over 30
! samtools view -b -q 30 ./P4/3.sorted_alignments.bam NC_012920.1:0-10000 > ./P4/4.filtered_alignments.bam   

In [39]:
! ls ./P4/

1.aligned_reads.sam            mito_human_index.4.bt2
2.aligned_reads.bam            mito_human_index.rev.1.bt2
3.sorted_alignments.bam        mito_human_index.rev.2.bt2
3.sorted_alignments.bam.bai    mito_human_ref_nc_012920.fasta
4.filtered_alignments.bam      mito_human_ref_nc_012920.gb
Archive                        mito_reads.tar.gz
mito_human_index.1.bt2         mito_reads_pe1.fastq.gz
mito_human_index.2.bt2         mito_reads_pe2.fastq.gz
mito_human_index.3.bt2


** **

*Bam statistics*

**flagstat**

Provides counts for each of 13 categories based primarily on bit flags in the FLAG field. Each category in the output is broken down into QC pass and QC fail. In the default output format, these are presented as `#PASS + #FAIL` followed by a description of the category.

* The first row of output gives the total number of reads that are QC pass and fail (according to flag bit 0x200). For example: `122 + 28 in total (QC-passed reads + QC-failed reads)`. Which would indicate that there are a total of 150 reads in the input file, 122 of which are marked as QC pass and 28 of which are marked as "not passing quality controls"

* Following this, additional categories are given for reads which are:
  * secondary: 0x100 bit set
  * supplementary: 0x800 bit set
  * duplicates: 0x400 bit set
  * mapped: 0x4 bit not set
  * paired in sequencing: 0x1 bit set
  * read1: both 0x1 and 0x40 bits set
  * read2: both 0x1 and 0x80 bits set
  * properly paired: both 0x1 and 0x2 bits set and 0x4 bit not set
  * with itself and mate mapped: 0x1 bit set and neither 0x4 nor 0x8 bits set
  * singletons: both 0x1 and 0x8 bits set and bit 0x4 not set

* Two rows are given that additionally filter on the reference name (`RNAME`), mate reference name (`MRNM`), and mapping quality (MAPQ) fields:
  * with mate mapped to a different chr: 0x1 bit set and neither 0x4 nor 0x8 bits set and MRNM not equal to RNAME
  * with mate mapped to a different chr (mapQ>=5): 0x1 bit set and neither 0x4 nor 0x8 bits set and MRNM not equal to RNAME and MAPQ >= 5

More info: http://www.htslib.org/doc/samtools-flagstat.html

In [40]:
! samtools flagstat ./P4/4.filtered_alignments.bam

11989 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
11989 + 0 mapped (100.00% : N/A)
11989 + 0 paired in sequencing
6005 + 0 read1
5984 + 0 read2
6021 + 0 properly paired (50.22% : N/A)
11963 + 0 with itself and mate mapped
26 + 0 singletons (0.22% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


**stats** 

Collects statistics from BAM files and outputs in a text format. The output can be visualized graphically using `plot-bamstats`.

More info: http://www.htslib.org/doc/samtools-stats.html

In [42]:
! samtools stats ./P4/3.sorted_alignments.bam > ./P4/5.sorted_alignments.stats

In [1]:
! plot-bamstats -p ./P4/6.stat_output ./P4/5.sorted_alignments.stats

In [2]:
! ls ./P4/

1.aligned_reads.sam            6.stat_output-quals.gp
2.aligned_reads.bam            6.stat_output-quals.png
3.sorted_alignments.bam        6.stat_output-quals2.gp
3.sorted_alignments.bam.bai    6.stat_output-quals2.png
4.filtered_alignments.bam      6.stat_output-quals3.gp
5.sorted_alignments.stats      6.stat_output-quals3.png
6.stat_output-acgt-cycles.gp   6.stat_output.html
6.stat_output-acgt-cycles.png  Archive
6.stat_output-coverage.gp      mito_human_index.1.bt2
6.stat_output-coverage.png     mito_human_index.2.bt2
6.stat_output-gc-content.gp    mito_human_index.3.bt2
6.stat_output-gc-content.png   mito_human_index.4.bt2
6.stat_output-indel-cycles.gp  mito_human_index.rev.1.bt2
6.stat_output-indel-cycles.png mito_human_index.rev.2.bt2
6.stat_output-indel-dist.gp    mito_human_ref_nc_012920.fasta
6.stat_output-indel-dist.png   mito_human_ref_nc_012920.gb
6.stat_output-insert-size.gp   mito_reads.tar.gz
6.stat_output-insert-size.png  mito_reads_pe1.fastq.gz
6.stat_output-quals-hm.

**idxstats** 
Retrieve and print stats in the index file corresponding to the input file. 

* Before calling idxstats, the input BAM file should be indexed by samtools index
* The output is TAB-delimited with each line consisting of 
  * Reference sequence name
  * Sequence length
  * Number mapped read-segments 
  * Number unmapped read-segments
* It is written to stdout

**Note** this may count reads multiple times if they are mapped more than once or in multiple fragments.

* The last line starts with a *star* and count the unmapped reads that are not located in any chromosome of the reference

More info: http://www.htslib.org/doc/samtools-idxstats.html

In [45]:
! samtools idxstats ./P4/3.sorted_alignments.bam

NC_012920.1	16569	19834	46
*	0	0	120


** ** 

*Coverage calculations* 

**bedtools** 
* Conda: https://bioconda.github.io/recipes/bedtools/README.html#package-bedtools
* Manual: https://bedtools.readthedocs.io/en/latest/

***coverage*** https://bedtools.readthedocs.io/en/latest/content/tools/coverage.html
* Blog: https://gettinggeneticsdone.blogspot.com/2014/03/visualize-coverage-exome-targeted-ngs-bedtools.html
* Tutorial: https://github.com/arq5x/bedtools-protocols/blob/master/bedtools.md

***genomecov*** https://bedtools.readthedocs.io/en/latest/content/tools/genomecov.html
* Answer: https://www.biostars.org/p/104063/

**Qualimap**
* Conda: https://bioconda.github.io/recipes/qualimap/README.html#package-qualimap
* Manual: http://qualimap.bioinfo.cipf.es/doc_html/analysis.html#global-plots

**Biopet-bamstats** - not recommended 
* Conda: https://bioconda.github.io/recipes/biopet-bamstats/README.html#package-biopet-bamstats
* Manual: http://bamstats.sourceforge.net/

**Mosdepth** - new
* Conda: https://bioconda.github.io/recipes/mosdepth/README.html#package-mosdepth
* Manual: https://github.com/brentp/mosdepth

**Deeptools** - seems cool
* Conda: https://bioconda.github.io/recipes/deeptools/README.html#package-deeptools
* Manual: https://deeptools.readthedocs.io/en/develop/content/list_of_tools.html

Useful info on how to do calculate the coverage: https://www.biostars.org/p/5165/
* Try Qualimap
* If not working samtools coverage http://www.htslib.org/doc/samtools-coverage.html
* Bedtools as option